In [1]:
import fix_yahoo_finance as yf
import pandas as pd
import requests
import pymongo
import datetime

In [2]:
# Extracted from https://www.nasdaq.com/screening/company-list.aspx
file_path = "Resources/companylist.csv"
df = pd.read_csv(file_path)
df = df[["Symbol","Name","LastSale","MarketCap","IPOyear","Sector","industry","Summary Quote"]]




df = df.loc[df["Sector"]=="Health Care"]
df = df.iloc[0:50,:]
start_date = "2004-05-03"
end_date = "2019-05-03"



df.head()

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,Summary Quote
1,MMM,3M Company,185.22,$106.77B,NaN,Health Care,Medical/Dental Instruments,https://www.nasdaq.com/symbol/mmm
8,AAC,"AAC Holdings, Inc.",1.97,$49.84M,2014.0,Health Care,Medical Specialities,https://www.nasdaq.com/symbol/aac
12,ABT,Abbott Laboratories,78.69,$138.22B,NaN,Health Care,Major Pharmaceuticals,https://www.nasdaq.com/symbol/abt
13,ABBV,AbbVie Inc.,78.71,$116.35B,2012.0,Health Care,Major Pharmaceuticals,https://www.nasdaq.com/symbol/abbv
69,ALC,Alcon Inc.,58.50,$28.59B,2019.0,Health Care,Ophthalmic Goods,https://www.nasdaq.com/symbol/alc


In [3]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [4]:
# Define database and collection
db = client.EcoParameters_db

db.StockPrice.drop()

collection = db.StockPrice



#df.Date = pd.to_datetime(df.Date)
#df.set_index('Date', inplace=True)
#df.resample('QS').sum()


ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it

In [ ]:
tickers = list(df["Symbol"])



for ticker in tickers:
    #try:
     #   data = yf.download(ticker, start=start_date, end=end_date)    
     #   flag = True
    #except Exception as e:
    #    print(e)
    #    flag = False
    
    data = yf.download(ticker, start=start_date, end=end_date)
    
    
    try:
        data = data.resample('QS').mean()
        data.reset_index(inplace=True)
        data["Ticker"] = ticker
        data = data[["Ticker","Date","Close","Volume"]]
        list_dict_prices_stock = []
        for item, row in data.iterrows():
            dict_row ={
                "Ticker" : row["Ticker"],
                "Date" : row["Date"].strftime('%Y-%m-%d'),
                "Close" : row["Close"],
                "Volume" : row["Volume"]
                }
            list_dict_prices_stock.append(dict_row)
        db.StockPrice.insert_many(list_dict_prices_stock)
    except Exception as e:
        print(e)
        
    
    

In [ ]:
list_price = list(db.StockPrice.find({"Ticker":"ACN"}))
print(list_price)